In [1]:
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
import seaborn as sns
import pandas as pd
from functools import reduce
#pip install getFamaFrenchFactors
import getFamaFrenchFactors as gff
#pip install fredapi
from fredapi import Fred
import matplotlib.pyplot as plt
from datetime import datetime
import statsmodels.api as sm

### Fama French

In [2]:
data_ff5 = pd.read_csv('F-F_Research_Data_5_Factors_2x3_daily.csv', skiprows=3)

In [3]:
data_ff5.rename(columns = {'Unnamed: 0':'Date'},inplace=True)

In [4]:
data_ff5['Date']=data_ff5['Date'].apply(str)


In [5]:
data_ff5.head()

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,19630701,-0.67,0.02,-0.35,0.03,0.13,0.012
1,19630702,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,19630703,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,19630705,0.40,0.09,-0.28,0.07,-0.30,0.012
4,19630708,-0.63,0.07,-0.20,-0.27,0.06,0.012


In [6]:
data_ff5.reset_index(drop = True, inplace= True)
data_ff5 = data_ff5[(data_ff5.Date > '20200101') & (data_ff5.Date <= '20231231')]
data_ff5['Date']=pd.to_datetime(data_ff5['Date'])
data_ff5

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
14223,2020-01-02,0.86,-0.97,-0.34,0.24,-0.22,0.006
14224,2020-01-03,-0.67,0.30,0.00,-0.14,-0.10,0.006
14225,2020-01-06,0.36,-0.20,-0.55,-0.17,-0.26,0.006
14226,2020-01-07,-0.19,-0.03,-0.25,-0.12,-0.25,0.006
14227,2020-01-08,0.47,-0.17,-0.64,-0.19,-0.17,0.006
...,...,...,...,...,...,...,...
15183,2023-10-25,-1.58,-0.02,0.84,1.50,0.20,0.021
15184,2023-10-26,-1.15,1.19,1.66,-0.76,0.48,0.021
15185,2023-10-27,-0.53,-0.58,-0.57,0.42,-0.96,0.021
15186,2023-10-30,1.15,-0.34,0.28,-0.14,0.17,0.021


In [7]:
df_ffs = data_ff5.set_index('Date')

In [8]:
df_ffs

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2020-01-02,0.86,-0.97,-0.34,0.24,-0.22,0.006
2020-01-03,-0.67,0.30,0.00,-0.14,-0.10,0.006
2020-01-06,0.36,-0.20,-0.55,-0.17,-0.26,0.006
2020-01-07,-0.19,-0.03,-0.25,-0.12,-0.25,0.006
2020-01-08,0.47,-0.17,-0.64,-0.19,-0.17,0.006
...,...,...,...,...,...,...
2023-10-25,-1.58,-0.02,0.84,1.50,0.20,0.021
2023-10-26,-1.15,1.19,1.66,-0.76,0.48,0.021
2023-10-27,-0.53,-0.58,-0.57,0.42,-0.96,0.021


### ADS Index

In [9]:
data_ads = pd.read_excel('ADS Index Most Current Vintage.xlsx')

In [10]:
data_ads

,Unnamed: 0,ADS_Index
0,1960:03:01,-0.572930
1,1960:03:02,-0.621487
2,1960:03:03,-0.666982
3,1960:03:04,-0.709432
4,1960:03:05,-0.748855
...,...,...
23282,2023:11:28,-0.147276
23283,2023:11:29,-0.143545
23284,2023:11:30,-0.140216
23285,2023:12:01,-0.137288


In [11]:
data_ads.rename(columns = {'Unnamed: 0':'Date'},inplace=True)

In [12]:
data_ads['Date'] = data_ads['Date'].str.replace(':', '-')

In [13]:
data_ads = data_ads[(data_ads.Date > '2020-01-01') & (data_ads.Date <= '2023-12-31')]

In [14]:
data_ads['Date'] = pd.to_datetime(data_ads['Date'])

In [15]:
df_ads = data_ads.set_index('Date')

In [16]:
df_ads

,ADS_Index
Date,
2020-01-02,-0.308743
2020-01-03,-0.292342
2020-01-04,-0.276362
2020-01-05,-0.260796
2020-01-06,-0.245729
...,...
2023-11-28,-0.147276
2023-11-29,-0.143545
2023-11-30,-0.140216


### Fred

In [17]:
fred = Fred(api_key='6fff19c7fff2c26131d75747d34452d6')

In [18]:
fred

In [19]:
varList = ['AMERIBOR', 'T10YIE', 'T5YIE', 'BAMLH0A0HYM2','BAMLH0A0HYM2EY', 
               'DGS10', 'DGS1', 'RIFSPPFAAD90NB', 'DCPN3M', 'DCPF1M', 'DCOILWTICO', 
               'DHHNGSP', 'USRECD', 'USRECDM', 'USRECDP'] 

In [20]:
SP500 = fred.get_series('SP500')
SP500.name = 'SP500'
df_fred = SP500

In [21]:
df_fred

2013-12-13    1775.32
2013-12-16    1786.54
2013-12-17    1781.00
2013-12-18    1810.65
2013-12-19    1809.60
               ...   
2023-12-06    4549.34
2023-12-07    4585.59
2023-12-08    4604.37
2023-12-11    4622.44
2023-12-12    4643.70
Name: SP500, Length: 2608, dtype: float64

In [22]:
# merge data series
for i in range(0, len(varList)):
    data = fred.get_series(varList[i])
    data.name = varList[i]
    df_fred = pd.merge(df_fred, data, left_index=True, right_index=True)

In [23]:
df_fred

,SP500,AMERIBOR,T10YIE,T5YIE,BAMLH0A0HYM2,BAMLH0A0HYM2EY,DGS10,DGS1,RIFSPPFAAD90NB,DCPN3M,DCPF1M,DCOILWTICO,DHHNGSP,USRECD,USRECDM,USRECDP
2015-12-11,2012.37,0.25333,1.46,1.17,7.10,8.75,2.13,0.68,0.63,NaN,NaN,35.65,1.79,0.0,0.0,0.0
2015-12-14,2021.94,0.21000,1.45,1.17,7.33,9.06,2.23,0.68,0.64,0.43,0.30,36.31,1.70,0.0,0.0,0.0
2015-12-15,2043.41,0.22000,1.48,1.22,7.09,8.86,2.28,0.69,0.56,NaN,0.32,37.32,1.66,0.0,0.0,0.0
2015-12-16,2073.07,0.22000,1.46,1.20,6.98,8.79,2.30,0.70,0.58,NaN,0.35,35.55,1.70,0.0,0.0,0.0
2015-12-17,2041.89,0.36500,1.47,1.21,7.01,8.79,2.24,0.69,0.55,0.43,0.35,34.98,1.79,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-29,4550.58,5.44731,2.21,2.14,3.80,8.18,4.27,5.12,5.33,NaN,5.32,77.56,2.70,0.0,0.0,0.0
2023-11-30,4567.80,5.46411,2.23,2.15,3.84,8.30,4.37,5.16,5.39,NaN,5.34,75.66,2.75,0.0,0.0,0.0
2023-12-01,4594.63,5.41739,2.22,2.12,3.87,8.19,4.22,5.05,5.37,NaN,5.33,73.70,2.63,0.0,0.0,0.0
2023-12-04,4569.78,5.43645,2.21,2.13,3.80,8.17,4.28,5.10,5.34,NaN,5.30,72.73,2.55,0.0,0.0,0.0


In [24]:
df_fred.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2083 entries, 2015-12-11 to 2023-12-05
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SP500           2009 non-null   float64
 1   AMERIBOR        2051 non-null   float64
 2   T10YIE          1997 non-null   float64
 3   T5YIE           1997 non-null   float64
 4   BAMLH0A0HYM2    2059 non-null   float64
 5   BAMLH0A0HYM2EY  2059 non-null   float64
 6   DGS10           1997 non-null   float64
 7   DGS1            1997 non-null   float64
 8   RIFSPPFAAD90NB  1701 non-null   float64
 9   DCPN3M          1443 non-null   float64
 10  DCPF1M          1269 non-null   float64
 11  DCOILWTICO      2000 non-null   float64
 12  DHHNGSP         2016 non-null   float64
 13  USRECD          2083 non-null   float64
 14  USRECDM         2083 non-null   float64
 15  USRECDP         2083 non-null   float64
dtypes: float64(16)
memory usage: 276.6 KB


### Merging Data

In [25]:
DATA_part1 = pd.merge(df_ffs, df_ads, left_index=True, right_index=True)

In [26]:
DATA = pd.merge(DATA_part1, df_fred,left_index=True, right_index=True)

In [27]:
DATA.reset_index(inplace=True)
DATA = DATA.rename(columns = {'index':'Date'})

In [28]:
DATA_part1

,Mkt-RF,SMB,HML,RMW,CMA,RF,ADS_Index
Date,,,,,,,
2020-01-02,0.86,-0.97,-0.34,0.24,-0.22,0.006,-0.308743
2020-01-03,-0.67,0.30,0.00,-0.14,-0.10,0.006,-0.292342
2020-01-06,0.36,-0.20,-0.55,-0.17,-0.26,0.006,-0.245729
2020-01-07,-0.19,-0.03,-0.25,-0.12,-0.25,0.006,-0.231156
2020-01-08,0.47,-0.17,-0.64,-0.19,-0.17,0.006,-0.217070
...,...,...,...,...,...,...,...
2023-10-25,-1.58,-0.02,0.84,1.50,0.20,0.021,-0.529790
2023-10-26,-1.15,1.19,1.66,-0.76,0.48,0.021,-0.521524
2023-10-27,-0.53,-0.58,-0.57,0.42,-0.96,0.021,-0.511475


### Collecting Lululemon stocks form yfinance

In [29]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,12,31)

LULU = yf.download('LULU',start_date ,end_date)
LULU.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [30]:
LULU

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,232.899994,233.470001,231.770004,233.419998,233.419998,1449300
1,2020-01-03,231.240005,234.500000,230.229996,232.639999,232.639999,1315400
2,2020-01-06,231.490005,235.529999,230.800003,235.429993,235.429993,1460700
3,2020-01-07,235.500000,236.850006,233.339996,233.699997,233.699997,1367900
4,2020-01-08,234.880005,238.009995,234.009995,237.270004,237.270004,1883900
...,...,...,...,...,...,...,...
990,2023-12-07,460.670013,466.630005,460.600006,464.670013,464.670013,3243500
991,2023-12-08,455.880005,493.799988,448.809998,489.640015,489.640015,7136900
992,2023-12-11,493.950012,507.440002,490.000000,502.739990,502.739990,2799000
993,2023-12-12,503.000000,504.399994,495.679993,503.649994,503.649994,1717100


In [31]:
LULU_new = pd.merge(LULU, DATA,on = 'Date',how='inner')

In [32]:
LULU_new

,Date,Open,High,Low,Close,Adj Close,Volume,Mkt-RF,SMB,HML,...,DGS10,DGS1,RIFSPPFAAD90NB,DCPN3M,DCPF1M,DCOILWTICO,DHHNGSP,USRECD,USRECDM,USRECDP
0,2020-01-02,232.899994,233.470001,231.770004,233.419998,233.419998,1449300,0.86,-0.97,-0.34,...,1.88,1.56,1.74,1.61,1.59,61.17,2.05,0.0,0.0,0.0
1,2020-01-03,231.240005,234.500000,230.229996,232.639999,232.639999,1315400,-0.67,0.30,0.00,...,1.80,1.55,1.70,1.60,1.55,63.00,2.06,0.0,0.0,0.0
2,2020-01-06,231.490005,235.529999,230.800003,235.429993,235.429993,1460700,0.36,-0.20,-0.55,...,1.81,1.54,1.68,1.60,1.55,63.27,2.10,0.0,0.0,0.0
3,2020-01-07,235.500000,236.850006,233.339996,233.699997,233.699997,1367900,-0.19,-0.03,-0.25,...,1.83,1.53,1.66,1.62,1.56,62.70,2.17,0.0,0.0,0.0
4,2020-01-08,234.880005,238.009995,234.009995,237.270004,237.270004,1883900,0.47,-0.17,-0.64,...,1.87,1.55,1.64,1.61,1.53,59.65,2.09,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,2023-10-25,401.000000,403.820007,396.119995,396.700012,396.700012,1309700,-1.58,-0.02,0.84,...,4.95,5.43,5.50,5.37,NaN,86.07,2.86,0.0,0.0,0.0
961,2023-10-26,393.179993,394.040009,386.000000,388.619995,388.619995,1837200,-1.15,1.19,1.66,...,4.86,5.39,NaN,5.40,NaN,83.80,2.87,0.0,0.0,0.0
962,2023-10-27,390.000000,392.209991,384.369995,386.820007,386.820007,1448600,-0.53,-0.58,-0.57,...,4.84,5.39,5.49,5.36,5.36,86.04,3.24,0.0,0.0,0.0
963,2023-10-30,388.440002,393.269989,387.750000,391.750000,391.750000,1005200,1.15,-0.34,0.28,...,4.88,5.41,NaN,NaN,5.40,83.03,3.17,0.0,0.0,0.0


In [33]:
# Yesterday's Close Price
LULU_new['Yest_Close']  = LULU_new['Adj Close'].shift(1)

In [34]:
# Tesla Stock Returns
LULU_new['stock_return'] = (LULU_new['Adj Close']-LULU_new['Yest_Close'])/(LULU_new['Yest_Close'])

In [35]:
LULU_new

,Date,Open,High,Low,Close,Adj Close,Volume,Mkt-RF,SMB,HML,...,RIFSPPFAAD90NB,DCPN3M,DCPF1M,DCOILWTICO,DHHNGSP,USRECD,USRECDM,USRECDP,Yest_Close,stock_return
0,2020-01-02,232.899994,233.470001,231.770004,233.419998,233.419998,1449300,0.86,-0.97,-0.34,...,1.74,1.61,1.59,61.17,2.05,0.0,0.0,0.0,NaN,NaN
1,2020-01-03,231.240005,234.500000,230.229996,232.639999,232.639999,1315400,-0.67,0.30,0.00,...,1.70,1.60,1.55,63.00,2.06,0.0,0.0,0.0,233.419998,-0.003342
2,2020-01-06,231.490005,235.529999,230.800003,235.429993,235.429993,1460700,0.36,-0.20,-0.55,...,1.68,1.60,1.55,63.27,2.10,0.0,0.0,0.0,232.639999,0.011993
3,2020-01-07,235.500000,236.850006,233.339996,233.699997,233.699997,1367900,-0.19,-0.03,-0.25,...,1.66,1.62,1.56,62.70,2.17,0.0,0.0,0.0,235.429993,-0.007348
4,2020-01-08,234.880005,238.009995,234.009995,237.270004,237.270004,1883900,0.47,-0.17,-0.64,...,1.64,1.61,1.53,59.65,2.09,0.0,0.0,0.0,233.699997,0.015276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,2023-10-25,401.000000,403.820007,396.119995,396.700012,396.700012,1309700,-1.58,-0.02,0.84,...,5.50,5.37,NaN,86.07,2.86,0.0,0.0,0.0,403.369995,-0.016536
961,2023-10-26,393.179993,394.040009,386.000000,388.619995,388.619995,1837200,-1.15,1.19,1.66,...,NaN,5.40,NaN,83.80,2.87,0.0,0.0,0.0,396.700012,-0.020368
962,2023-10-27,390.000000,392.209991,384.369995,386.820007,386.820007,1448600,-0.53,-0.58,-0.57,...,5.49,5.36,5.36,86.04,3.24,0.0,0.0,0.0,388.619995,-0.004632
963,2023-10-30,388.440002,393.269989,387.750000,391.750000,391.750000,1005200,1.15,-0.34,0.28,...,NaN,NaN,5.40,83.03,3.17,0.0,0.0,0.0,386.820007,0.012745


In [36]:
LULU_new = LULU_new[LULU_new['Yest_Close'].notna()]
LULU_new = LULU_new[LULU_new['Mkt-RF'].notna()]
LULU_new = LULU_new[LULU_new['SMB'].notna()]
LULU_new = LULU_new[LULU_new['HML'].notna()]
LULU_new = LULU_new[LULU_new['RF'].notna()]

### Correlation

In [ ]:
x = LULU_new.iloc[1:, 1:]
y = LULU_new.iloc[:, -2]
corrmat = LULU_new.corr()
top_corr_features = corrmat.index
plt.figure(figsize = (14,10))
hm = sns.heatmap(LULU_new[top_corr_features].corr(), annot = True, cmap = "YlOrRd")

In [38]:
X = LULU_new[['Mkt-RF', 'SMB', 'HML', 'ADS_Index']]/100
y = LULU_new['stock_return'] - LULU_new['RF']
X = sm.add_constant(X)

ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.458
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     202.4
Date:                Wed, 13 Dec 2023   Prob (F-statistic):          7.40e-126
Time:                        17:25:08   Log-Likelihood:                 2377.9
No. Observations:                 964   AIC:                            -4746.
Df Residuals:                     959   BIC:                            -4721.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0054      0.001     -8.158      0.0

In [39]:
X

,const,Mkt-RF,SMB,HML,ADS_Index
1,1.0,-0.0067,0.0030,0.0000,-0.002923
2,1.0,0.0036,-0.0020,-0.0055,-0.002457
3,1.0,-0.0019,-0.0003,-0.0025,-0.002312
4,1.0,0.0047,-0.0017,-0.0064,-0.002171
5,1.0,0.0065,-0.0071,-0.0048,-0.002035
...,...,...,...,...,...
960,1.0,-0.0158,-0.0002,0.0084,-0.005298
961,1.0,-0.0115,0.0119,0.0166,-0.005215
962,1.0,-0.0053,-0.0058,-0.0057,-0.005115
963,1.0,0.0115,-0.0034,0.0028,-0.004710


In [40]:
y

1     -0.009342
2      0.005993
3     -0.013348
4      0.009276
5     -0.011774
         ...   
960   -0.037536
961   -0.041368
962   -0.025632
963   -0.008255
964   -0.016584
Length: 964, dtype: float64

In [41]:
ff_model.params

const       -0.005423
Mkt-RF       1.138445
SMB          0.357755
HML         -0.466416
ADS_Index    0.002518
dtype: float64

LULU_new.to_csv('INFOP7374_FeatureMart4GS.csv', index=True)